## Import Libraries

In [5]:
import asyncio
import re
import pandas as pd
import os
from playwright.async_api import async_playwright
import nest_asyncio

# ✅ Fix "RuntimeError: asyncio.run() cannot be called from a running event loop"
nest_asyncio.apply()

## Function to Open Browser and Extract Data

In [7]:
async def scrape_page(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False, slow_mo=500)  # ✅ Opens the browser
        page = await browser.new_page()

        print(f"🔍 Opening: {url}")
        await page.goto(url, timeout=60000)

        # ✅ Wait for page to load
        await page.wait_for_selector("body", timeout=10000)
        await page.wait_for_timeout(5000)

        # ✅ Extract visible text (helps find emails & firm names)
        full_text = await page.evaluate("document.body.innerText")

        print(f"🔎 Extracted Text:\n{full_text[:500]}...\n{'='*50}\n")

        # ✅ Extract emails
        emails = extract_emails(full_text)
        # ✅ Extract firm name
        firm_name = extract_firm_name(full_text)

        await browser.close()
        return {"url": url, "emails": emails, "firm_name": firm_name}

## Extract Emails

In [9]:
def extract_emails(content):
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    emails = list(set(re.findall(email_pattern, content)))  # ✅ Removes duplicates

    print(f"📧 Extracted Emails: {emails}")
    return emails if emails else ["No Email Found"]

## Extract Company Name

In [11]:
def extract_firm_name(content):
    firm_name = "No Firm Name Found"

    # ✅ Use regex to capture text after "Contact details of"
    match = re.search(r"Contact details of\s*([^\n.]+)", content, re.IGNORECASE)

    if match:
        firm_name = match.group(1).strip()  # ✅ Get captured text after "Contact details of"

    print(f"🏢 Extracted Firm Name: {firm_name}")
    return firm_name

## Save Data

In [13]:
def save_to_excel(data, output_file="scraped_data.xlsx"):
    df = pd.DataFrame(data)

    if os.path.exists(output_file):
        os.remove(output_file)  # ✅ Prevents PermissionError

    df.to_excel(output_file, index=False)
    print(f"✅ Data saved to {output_file}")

## Main Scrapper

In [15]:
async def main_scraper(urls):
    all_data = []
    
    for url in urls:
        result = await scrape_page(url)
        all_data.append(result)
    
    save_to_excel(all_data)

## Call

In [17]:
urls = [
    "https://www.zaubacorp.com/MURLI-REALTORS-PRIVATE-LIMITED-U45201PN1995PTC085146",
    "https://www.zaubacorp.com/HI-5-BROADBAND-SOLUTION-PRIVATE-LIMITED-U74999PN2013PTC149284",
    "https://www.zaubacorp.com/STAR-MECH-CONTROLS-INDIA-PRIVATE-LTD-U74999MH1994PTC077723",
    "https://www.zaubacorp.com/STABLE-INDUSTRIES-PRIVATE-LIMITED-U74110PN2010PTC135601",
    "https://www.zaubacorp.com/VIRAJ-PROJECTS-PRIVATE-LIMITED-U45209PN2005PTC021092",
    "https://www.zaubacorp.com/LEARNSURE-AI-PRIVATE-LIMITED-U72100PN2014PTC151502",
    "https://www.zaubacorp.com/SHATAU-GLOBAL-MARKETING-PRIVATE-LIMITED-U74999PN2018PTC174502",
    "https://www.zaubacorp.com/SUYASH-AUTO-PRESS-COMPONENTS-AND-ASSEMBLYS-PVT-LTD-U50103MH1993PTC071402",
    "https://www.zaubacorp.com/ENRITA-S-SOLUTIONS-PRIVATE-LIMITED-U72900PN2001PTC157839",
    "https://www.zaubacorp.com/VIPIN-FOREX-PRIVATE-LIMITED-U67190PN2004PTC018851",
    "https://www.zaubacorp.com/FORI-AUTOMATION-INDIA-PRIVATE-LIMITED-U93090PN2007PTC131552",
    "https://www.zaubacorp.com/MIDDSHINE-TECHNOLOGIS-PRIVATE-LIMITED-U74110PN2013PTC147010",
    "https://www.zaubacorp.com/GLEANORE-INDIA-PRIVATE-LIMITED-U74999PN2018PTC174839",
    "https://www.zaubacorp.com/AIRCARE-SYSTEM-AND-SOLUTION-INDIA-PRIVATE-LIMITED-U74999PN2017PTC170048",
    "https://www.zaubacorp.com/TONTECH-ENGINEERS-PRIVATE-LIMITED-U45400PN2010PTC155142",
    "https://www.zaubacorp.com/YASHANAND-INFRA-PRIVATE-LIMITED-U45200PN2015PTC154053",
    "https://www.zaubacorp.com/CONNVERTEX-TECHNOLOGIES-INDIA-PRIVATE-LIMITED-U72900PN2011FTC139378",
    "https://www.zaubacorp.com/NEW-BALANCE-INDIA-PRIVATE-LIMITED-U51909PN2022FTC207710",
    "https://www.zaubacorp.com/MAGIC-SOFTWARE-ENTERPRISES-INDIA-PRIVATE-LIMITED-U72200MH1997PTC112303",
    "https://www.zaubacorp.com/ALLIED-TRANSLINK-PRIVATE-LIMITED-U60200PN2016PTC158759",
    "https://www.zaubacorp.com/SAKA-ENGINEERING-SYSTEMS-PRIVATE-LIMITED-U74900PN2010PTC135874",
    "https://www.zaubacorp.com/DEMECH-OVERSEAS-CONSTRUCTION-PVT-LTD-U45309PN1978PTC020599",
    "https://www.zaubacorp.com/M-B-LANDMARKS-PRIVATE-LIMITED-U45400PN2015PTC157183",
    "https://www.zaubacorp.com/KAYSER-AUTOMOTIVE-SYSTEMS-INDIA-PRIVATE-LIMITED-U74900PN2013FTC149648",
    "https://www.zaubacorp.com/DOUBLE-EIGHTS-VENTURE-PRIVATE-LIMITED-U74999PN2018PTC203623",
    "https://www.zaubacorp.com/LINQ-THERAPEUTICS-PRIVATE-LIMITED-U74999PN2015PTC156595",
    "https://www.zaubacorp.com/ZAHEER-VAKIL-ESTATES-PRIVATE-LIMITED-U45200PN2002PTC214667",
    "https://www.zaubacorp.com/KESHAR-AGROTECH-PRIVATE-LIMITED-U01111PN2000PTC014790",
    "https://www.zaubacorp.com/AJINKYA-MOTORS-PVT-LTD-U01100MH1987PTC044573",
    "https://www.zaubacorp.com/INDRAYANI-FOODS-LTD-U15543MH1994PLC076178",
    "https://www.zaubacorp.com/MRC-LOGISTICS-INDIA-PRIVATE-LIMITED-U63090PN2002PTC017316",
    "https://www.zaubacorp.com/KALYANI-SKILLS-PRIVATE-LIMITED-U80903PN2012PTC143985",
    "https://www.zaubacorp.com/SAI-SERVICE-SPARES-AND-ACCESSORIES-PRIVATE-LIMITED-U23300PN1991PTC062654",
    "https://www.zaubacorp.com/MANCHAR-DIAGNOSTICS-CENTRE-PRIVATE-LIMITED-U74999PN2016PTC164868",
    "https://www.zaubacorp.com/MANAS-HOLIDAY-RESORTS-PRIVATE-LIMITED-U55901PN1998PTC013011",
    "https://www.zaubacorp.com/TAMARIND-INSURANCE-BROKERS-PRIVATE-LIMITED-U67100PN2022PTC215299",
    "https://www.zaubacorp.com/SAAKETH-ELECTROCIRCUIT-PRIVATE-LIMITED-U74999PN2019PTC182897",
    "https://www.zaubacorp.com/TECHENCE-IT-SOLUTIONS-PRIVATE-LIMITED-U72200PN2012PTC142133",
    "https://www.zaubacorp.com/KESAR-FOREX-PRIVATE-LIMITED-U65999PN2005PTC020265",
    "https://www.zaubacorp.com/VISWAKARMA-BUILDTECH-INDIA-PRIVATE-LIMITED-U45100PN2009PTC148904",
    "https://www.zaubacorp.com/THERMON-INDIA-PRIVATE-LIMITED-U99999MH1983PTC030628",
    "https://www.zaubacorp.com/DATTATRAYAPUSHPALATA-PROPERTIES-PRIVATE-LIMITED-U70109PN2005PTC021227",
    "https://www.zaubacorp.com/PIMPRI-CHINCHWAD-MEDIA-COMMUNICATIONS-PRIVATE-LIMITED-U32201PN2011PTC142898",
    "https://www.zaubacorp.com/ACE-ELECTRONICS-PRIVATE-LIMITED-U72200PN2013PTC147513",
    "https://www.zaubacorp.com/ROHAN-CORRUPACK-PRIVATE-LIMITED-U21023PN2011PTC139857",
    "https://www.zaubacorp.com/HUNS-DIGITAL-HOMES-PRIVATE-LIMITED-U51505PN2002PTC017218",
    "https://www.zaubacorp.com/EMPORIS-AUTOACCESS-PRIVATE-LIMITED-U74900PN2011PTC141669",
    "https://www.zaubacorp.com/D-B-DEVELOPERS-LIMITED-U45202PN1999PLC013957",
    "https://www.zaubacorp.com/MAGNET-AUTOTECH-SOLUTIONS-PRIVATE-LIMITED-U74999PN2016PTC164819",
    "https://www.zaubacorp.com/ENRICH-AGRO-PROCESSING-INDUSTRY-PRIVATE-LIMITED-U74990PN2015PTC157367",
    "https://www.zaubacorp.com/PURE-HEALTH-PRODUCTS-PRIVATE-LIMITED-U85199PN1988PTC047757",
    "https://www.zaubacorp.com/DEEKSHA-EXPORTS-PRIVATE-LIMITED-U51229PN2004PTC019715",
    "https://www.zaubacorp.com/ASPIRE-KNOWLEDGE-AND-SKILLS-INDIA-PRIVATE-LIMITED-U80900PN2017PTC168844",
    "https://www.zaubacorp.com/TRISHLA-HOTELS-PVT-LTD-U55101PN1982PTC026284",
    "https://www.zaubacorp.com/BAMBOLI-AGENCIES-PRIVATE-LIMITED-U51505PN2004PTC020027",
    "https://www.zaubacorp.com/RAMA-AUTOMOBILES-AGENCY-PRIVATE-LIMITED-U50401PN1993PTC075477",
    "https://www.zaubacorp.com/ABHINAV-NIRMAN-PRIVATE-LIMITED-U45200PN2012PTC145460",
    "https://www.zaubacorp.com/PUSHPAM-PLAZA-PRIVATE-LIMITED-U45309PN1995PTC091617",
    "https://www.zaubacorp.com/APARNA-SERVICES-PRIVATE-LIMITED-U51909MH1987PTC044190",
    "https://www.zaubacorp.com/G-M-KENJALE-CONSTRUCTIONS-PVT-LTD-U95202PN1993PTC073644",
    "https://www.zaubacorp.com/SANCHETI-FAMILY-FOODS-INDIA-PRIVATE-LIMITED-U15135PN1998PTC012638",
    "https://www.zaubacorp.com/CANPEX-CHEMICALS-PRIVATE-LIMITED-U24121PN1985PTC035089",
    "https://www.zaubacorp.com/PRIMA-GEAR-CUTTING-TOOLS-PRIVATE-LIMITED-U28113MH1994PTC082166",
    "https://www.zaubacorp.com/CRESCENT-FUSES-PRIVATE-LIMITED-U31300PN1998PTC112811",
    "https://www.zaubacorp.com/SSM-ORGANIC-PRIVATE-LIMITED-U40100PN2018PTC174717",
    "https://www.zaubacorp.com/YASH-MIRAJ-DEVELOPMENT-COMPANY-PRIVATE-LIMITED-U45200PN2014PTC151175",
    "https://www.zaubacorp.com/URVINA-INDIA-PRIVATE-LIMITED-U74900PN2013FTC149313",
    "https://www.zaubacorp.com/WIZKIDSCARNIVAL-TECHNOLOGIES-PRIVATE-LIMITED-U93090PN2020PTC191901",
    "https://www.zaubacorp.com/LASERLINE-DIODE-LASER-TECHNOLOGY-PRIVATE-LIMITED-U74990PN2019FTC184521",
    "https://www.zaubacorp.com/FOLDY-PACKAGING-PRIVATE-LIMITED-U36990PN2021PTC201701",
    "https://www.zaubacorp.com/CEREBLE-PRIVATE-LIMITED-U72900PN2019PTC185985",
    "https://www.zaubacorp.com/VASPAN-HEALING-PRIVATE-LIMITED-U74900PN2015PTC154772",
    "https://www.zaubacorp.com/TAMBOLI-ENGINEERS-PRIVATE-LIMITED-U00359PN2006PTC022255",
    "https://www.zaubacorp.com/KAPEESHWARA-PLASTICS-PRIVATE-LIMITED-U74999PN2016PTC166630",
    "https://www.zaubacorp.com/MAHARASHTRA-ERECTORS-PRIVATE-LIMITED-U45201PN1991PTC062960",
    "https://www.zaubacorp.com/SARDESAI-FINANCE-PRIVATE-LIMITED-U67190MH1991PTC061030",
    "https://www.zaubacorp.com/SAWALKA-TOOLS-AND-MACHINES-PRIVATE-LIMITED-U99999MH1985PTC070692",
    "https://www.zaubacorp.com/BAJAJ-ALLIANZ-STAFFING-SOLUTIONS-LIMITED-U74900PN2015PLC154364",
    "https://www.zaubacorp.com/ELVIN-FURNITURE-PRIVATE-LIMITED-U79191MH1989PTC054755",
    "https://www.zaubacorp.com/NASAN-MEDICAL-SYSTEMS-LIMITED-U33112PN2000PLC018531",
    "https://www.zaubacorp.com/SPARTAN-FINVOYANCE-PRIVATE-LIMITED-U74999PN2019PTC181605",
    "https://www.zaubacorp.com/MAXIM-PROMOTERS-PRIVATE-LIMITED-U45203PN2000PTC014424",
    "https://www.zaubacorp.com/INTEGRATED-ACTIVE-MONITORING-PRIVATE-LIMITED-U74999PN2016PTC167219",
    "https://www.zaubacorp.com/POONAWALLA-FINVEST-AND-AGRO-PRIVATE-LIMITED-U65993PN1989PTC051541",
    "https://www.zaubacorp.com/FABRI-TEK-EQUIPMENTS-PRIVATE-LIMITED-U74900PN2014PTC150193",
    "https://www.zaubacorp.com/SUAN-DIGITAL-IMAGES-PRIVATE-LIMITED-U74999PN2010PTC135960",
    "https://www.zaubacorp.com/OPULENT-INFOTECH-PRIVATE-LIMITED-U72200PN2015PTC154415",
    "https://www.zaubacorp.com/SHREE-STHAPATYA-PRIVATE-LIMITED-U45201PN1994PTC081659",
    "https://www.zaubacorp.com/S-K-SANSKRITI-MOTORS-PRIVATE-LIMITED-U50200PN2014PTC151702",
    "https://www.zaubacorp.com/YOGESH-FINANCIAL-CONSULTANTS-PRIVATE-LIMITED-U65993PN1987PTC045285",
    "https://www.zaubacorp.com/BAYAS-HOSPITALITY-AND-CONSULTING-SERVICES-PRIVATE-LIMITED-U55209PN2023PTC217886",
    "https://www.zaubacorp.com/JANAK-MERCHANT-SECURITIES-PRIVATE-LIMITED-U67120MH1997PTC111655",
    "https://www.zaubacorp.com/UNIQUE-VALVES-PRIVATE-LIMITED-U74920MH1983PTC030275",
    "https://www.zaubacorp.com/LINC-WISE-SOFTWARE-PRIVATE-LIMITED-U72200PN1996PTC014624",
    "https://www.zaubacorp.com/TALERA-HOTELIERS-PVT-LTD-U55101MH1979PTC021337",
    "https://www.zaubacorp.com/ZIRKEL-INFRACA-PRIVATE-LIMITED-U74999PN2015FTC157225",
    "https://www.zaubacorp.com/VIRANI-IRON-PRIVATE-LIMITED-U51909PN2018PTC175867",
    "https://www.zaubacorp.com/CATTERFLY-TRAVEL-PRIVATE-LIMITED-U63090PN2020FTC189381",
    "https://www.zaubacorp.com/SHIVNERI-MUTUAL-BENEFIT-NIDHI-LIMITED-U65923PN2015PLC155242",
    "https://www.zaubacorp.com/ASHWAMEGH-ENTERPRISES-PRIVATE-LIMITED-U74140PN1987PTC043715",
    "https://www.zaubacorp.com/PRATIBIMB-FLEX-PACK-PRIVATE-LIMITED-U74950PN2006PTC129233",
    "https://www.zaubacorp.com/PANCHGANGA-INFRA-PRIVATE-LIMITED-U45200PN2003PTC017729",
    "https://www.zaubacorp.com/MAEE-HOTELS-AND-HOSPITALITIES-PRIVATE-LIMITED-U55204PN2020PTC193563",
    "https://www.zaubacorp.com/TEJAL-TRANSMISSION-PRIVATE-LIMITED-U34300MH1990PTC058014",
    "https://www.zaubacorp.com/SAMVED-ENERGY-SYSTEMS-PRIVATE-LIMITED-U40102PN2008PTC132743",
    "https://www.zaubacorp.com/VOHRA-AUTO-DISTRIBUTORS-PRIVATE-LIMITED-U50401PN1995PTC092792",
    "https://www.zaubacorp.com/PRIMA-PRIVATE-LIMITED-U21012MH1974PTC017894",
    "https://www.zaubacorp.com/SCM-SCREW-PRODUCTS-PRIVATE-LIMITED-U27310PN2017PTC172188",
    "https://www.zaubacorp.com/VACUUM-PLANT-AND-INSTRUMENTS-MANUFACTURING-COMPANY-LIMITED-U29120PN1965PLC013099",
    "https://www.zaubacorp.com/SAVAN-TOOLS-AND-ANCILLARIES-PRIVATE-LIMITED-U29299PN2008PTC131749",
    "https://www.zaubacorp.com/PASHANKAR-AUTO-PRIVATE-LIMITED-U34102PN2001PTC015973",
    "https://www.zaubacorp.com/ASTUTE-ENGINEERING-SERVICES-PRIVATE-LIMITED-U74200PN2009PTC134749",
    "https://www.zaubacorp.com/SHREE-GAJLAXMI-HOTELS-PRIVATE-LIMITED-U55101PN2002PTC017294",
    "https://www.zaubacorp.com/WIKKERWORK-SOFTWARE-PRIVATE-LIMITED-U72200PN2015PTC154953",
    "https://www.zaubacorp.com/HORIZON-FITNESS-PRIVATE-LIMITED-U74900PN2012PTC143285",
    "https://www.zaubacorp.com/FLY-LEMON-TRAVEL-AND-FOREX-PRIVATE-LIMITED-U63030PN2016PTC158995",
    "https://www.zaubacorp.com/VASU-LAYER-BREEDER-FARM-PRIVATE-LIMITED-U01222PN2006PTC128745",
    "https://www.zaubacorp.com/BRICKETC-PRIVATE-LIMITED-U80903PN2021PTC205854",
    "https://www.zaubacorp.com/AYODHYA-INDUSTRIES-PRIVATE-LIMITED-U51909PN2019PTC181586",
    "https://www.zaubacorp.com/BALAJI-AUTO-INDUSTRIES-PRIVATE-LIMITED-U35900PN2021PTC201176",
    "https://www.zaubacorp.com/ATHARVA-NATURE-HEALTHCARE-PRIVATE-LIMITED-U74999PN2014PTC150341",
    "https://www.zaubacorp.com/REGAL-INDIA-LOGISTICS-PRIVATE-LIMITED-U63030PN2010PTC136199",
    "https://www.zaubacorp.com/MAHAMUNI-ENTERPRISES-PRIVATE-LIMITED-U74900PN2014PTC151190",
    "https://www.zaubacorp.com/SOFTZEAL-TECHNOLOGY-PRIVATE-LIMITED-U72200PN2011PTC141048",
    "https://www.zaubacorp.com/AGNI-SOLAR-SYSTEMS-PRIVATE-LIMITED-U72100PN2000PTC015021",
    "https://www.zaubacorp.com/MAAN-TRADING-PRIVATE-LIMITED-U74900PN2012PTC144905",
    "https://www.zaubacorp.com/ANW-STUDIOS-PRIVATE-LIMITED-U92419PN2021PTC198358",
    "https://www.zaubacorp.com/DESIGN-MEDIA-AND-EDUTAINMENT-SOLUTIONS-PRIVATE-LIMITED-U80301PN2015PTC156693",
    "https://www.zaubacorp.com/EXABYTE-INFOTECH-PRIVATE-LIMITED-U72900PN2018PTC177826",
    "https://www.zaubacorp.com/BHOSCH-ELECTRONICS-APPLIED-TECHNOLOGY-PRIVATE-LIMITED-U36990PN2020PTC191435",
    "https://www.zaubacorp.com/ALPHABRICKS-TECHNOLOGIES-PRIVATE-LIMITED-U74999PN2010PTC137155",
    "https://www.zaubacorp.com/SWARN-SUPER-SPECIALTY-HOSPITAL-PRIVATE-LIMITED-U74220PN2016PTC164647",
    "https://www.zaubacorp.com/JAGDISH-HOTELS-AND-INVESTMENTS-PRIVATE-LIMITED-U55101PN1997PTC112323",
    "https://www.zaubacorp.com/MY-WINGS-PRIVATE-LIMITED-U34100PN2010PTC135801",
    "https://www.zaubacorp.com/FITMAT-SMART-SOLUTIONS-PRIVATE-LIMITED-U74999PN2019FTC183493",
    "https://www.zaubacorp.com/PRATHAM-ENERGY-SYSTEMS-PRIVATE-LIMITED-U51909PN2020PTC190288",
    "https://www.zaubacorp.com/CLIMOTION-INDIA-PRIVATE-LIMITED-U74999PN2018PTC178189",
    "https://www.zaubacorp.com/KAUSTUBH-ROOFING-INDUSTRIES-PRIVATE-LIMITED-U74900PN2015PTC155229",
    "https://www.zaubacorp.com/APPL-DEFENCE-AND-TELECOMMUNICATIONS-PRIVATE-LIMITED-U74999PN2018PTC179250",
    "https://www.zaubacorp.com/YOGI-WIRES-PRIVATE-LIMITED-U74900PN2008PTC132493",
    "https://www.zaubacorp.com/THAI-INDIA-EXPRESS-PRIVATE-LIMITED-U60232PN2017PTC169686",
    "https://www.zaubacorp.com/LFS-COMMODITIES-PRIVATE-LIMITED-U74999PN2006PTC147846",
    "https://www.zaubacorp.com/EMERGE-RESOURCES-PRIVATE-LIMITED-U51909PN2016PTC158468",
    "https://www.zaubacorp.com/GOLDEN-GLADES-LIMITED-U70101MH1990PLC056756",
    "https://www.zaubacorp.com/GIG-PRODUCTION-PRIVATE-LIMITED-U72900PN2016PTC165995",
    "https://www.zaubacorp.com/NASHIK-ONLINE-LIMITED-U72900PN2012PLC142788",
    "https://www.zaubacorp.com/GRASS-INDIA-FITTINGS-PRIVATE-LIMITED-U36994PN2017FTC167992",
    "https://www.zaubacorp.com/COTMAC-INDUSTRIAL-TRADING-PRIVATE-LIMITED-U51909PN2010PTC136352",
    "https://www.zaubacorp.com/SIDDHII-SLIDES-PRIVATE-LIMITED-U74999PN2016PTC165625",
    "https://www.zaubacorp.com/PLAYER-X-MOBILE-ENTERTAINMENT-INDIA-PRIVATE-LIMITED-U72300PN2007PTC142972"
]

await main_scraper(urls)  # ✅ Works in Jupyter Notebook

🔍 Opening: https://www.zaubacorp.com/MURLI-REALTORS-PRIVATE-LIMITED-U45201PN1995PTC085146
🔎 Extracted Text:
 My Cart
Sign In
Sign Up
Company
Director
Trademark
Address
Submit
Browse Companies by Activity, Age and Location
Company Information
 
Directors
 
Contact Information
 
Legal Check
 
Financial Reports
 
Investors
MURLI REALTORS PRIVATE LIMITED
As on: 2024-06-26

MURLI REALTORS PRIVATE LIMITED (CIN: U45201PN1995PTC085146) is a Private company incorporated on 01 Dec 1995. It is classified as Non-government company and is registered at Registrar of Companies, Pune. Its authorized share capital is ...

📧 Extracted Emails: ['arsakpal@manikchandgroup.com']
🏢 Extracted Firm Name: MURLI REALTORS
🔍 Opening: https://www.zaubacorp.com/HI-5-BROADBAND-SOLUTION-PRIVATE-LIMITED-U74999PN2013PTC149284
🔎 Extracted Text:
 My Cart
Sign In
Sign Up
Company
Director
Trademark
Address
Submit
Browse Companies by Activity, Age and Location
Company Information
 
Directors
 
Contact Information
 
Legal Che